In [1]:
import numpy as np
import scipy.ndimage as ndimage

# Segmentation filter
The original implementation is:

In [2]:
def generateTileOriginal(kernel, image, variance, threshold=.5):
    output = np.zeros(image.shape)
    hx = kernel.shape[0] // 2
    hy = kernel.shape[1] // 2
    width = image.shape[0]
    height = image.shape[1]

    for iy in range(0, height):
        for ix in range(0, width):
            total = 0.
            total_ignoring_threshold = 0.
            conv_weight = 0.
            conv_weight_ignoring_threshold = 0.
            if variance[ix, iy] < threshold:
                for cy in range(0, kernel.shape[1]):
                    for cx in range(0, kernel.shape[0]):
                        x2 = ix + cx - hx
                        y2 = iy + cy - hy

                        if x2 >= 0 and x2 < width and y2 >= 0 and y2 < height:
                            if variance[x2,y2] < threshold:
                                total += image[x2, y2] * kernel[cx, cy]
                                conv_weight += kernel[cx, cy]
                            total_ignoring_threshold += image[x2, y2] * kernel[cx, cy]
                            conv_weight_ignoring_threshold += kernel[cx, cy]

                if conv_weight > 0:
                    output[ix, iy] = total / conv_weight
                else:
                    output[ix, iy] = total_ignoring_threshold / conv_weight_ignoring_threshold

    return output

### A first rewrite
First thing to notice is that the variance map is used only in two conditionals: lines 14 and 21. Therefore, we could generate an image where pixels are 1 if the variance is below the threshold, or 0 if above, and replace

```python
if variance[x2,y2] < threshold:
```

with 

```python
if mask[x2,y2]
``` 

If we inspect closer where this conditional would be used, we can see that the effect is, in the inner `if`, equivalent to add 0 when the mask is 0, or the value of the image multiplied by the kernel when it is 1; and similarly for the weight addition. So we can get rid of the conditional, and rewrite as:

```python
total += image[x2, y2] * kernel[cx, cy] * mask[x2, y2]
conv_weight += kernel[cx, cy] * mask[x2, y2]
```

The variables `total_ignoring_threshold` and `conv_weight_ignoring_threshold` are kind of pointless since the outer conditional was added, as, at least, the central pixel will have a variance lower than the threshold. So conv_weight will always be greater than 0.

In [3]:
def generateTileVersionA(kernel, image, variance, threshold=.5):
    output = np.zeros(image.shape)
    hx = kernel.shape[0] // 2
    hy = kernel.shape[1] // 2
    width = image.shape[0]
    height = image.shape[1]
    mask = (variance < threshold).astype(np.float)

    for iy in range(0, height):
        for ix in range(0, width):
            total = 0.
            conv_weight = 0.
            if mask[ix, iy]:
                for cy in range(0, kernel.shape[1]):
                    for cx in range(0, kernel.shape[0]):
                        x2 = ix + cx - hx
                        y2 = iy + cy - hy

                        if x2 >= 0 and x2 < width and y2 >= 0 and y2 < height:
                            total += image[x2, y2] * kernel[cx, cy] * mask[x2, y2]
                            conv_weight += kernel[cx, cy] * mask[x2, y2]

                output[ix, iy] = total / conv_weight

    return output

Let's verify they give the same result.

In [4]:
img = np.arange(25).reshape(5,-1).astype(np.float)
var = np.random.random((5,5))
kernel=np.array([[0,1,0],[1,2,1],[0,1,0]]).astype(np.float)
threshold = 0.5

In [5]:
img

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]])

In [6]:
var

array([[0.26966613, 0.51548509, 0.82427199, 0.11087943, 0.27400985],
       [0.5773263 , 0.50657198, 0.34718092, 0.59590405, 0.30646345],
       [0.7754545 , 0.44818602, 0.98572844, 0.0455693 , 0.27501566],
       [0.70661158, 0.95080935, 0.54487754, 0.97771282, 0.44513659],
       [0.03833808, 0.01719552, 0.29122308, 0.25883254, 0.59640376]])

In [7]:
kernel

array([[0., 1., 0.],
       [1., 2., 1.],
       [0., 1., 0.]])

In [8]:
generateTileOriginal(kernel, img, var, threshold)

array([[ 0.        ,  0.        ,  0.        ,  3.33333333,  5.        ],
       [ 0.        ,  0.        ,  7.        ,  0.        ,  9.        ],
       [ 0.        , 11.        ,  0.        , 13.33333333, 13.8       ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 17.33333333],
       [20.33333333, 21.        , 22.        , 22.66666667,  0.        ]])

In [9]:
generateTileVersionA(kernel, img, var, threshold)

array([[ 0.        ,  0.        ,  0.        ,  3.33333333,  5.        ],
       [ 0.        ,  0.        ,  7.        ,  0.        ,  9.        ],
       [ 0.        , 11.        ,  0.        , 13.33333333, 13.8       ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 17.33333333],
       [20.33333333, 21.        , 22.        , 22.66666667,  0.        ]])

They do match.

### Second rewrite
Can we get rid of the other conditional, and use the mask instead? Well, that `if` is basically setting a pixel to 0 if the mask is 0, and to the convolved value otherwise. We could get rid of it, and move the if (or the application of the mask) outside.

If we do this, we need to put a check for `conv_weight`, as it may be be 0 now! Again, if the mask is not 0, then it will have a value.

In [10]:
def generateTileVersionB(kernel, image, variance, threshold=.5):
    output = np.zeros(image.shape)
    hx = kernel.shape[0] // 2
    hy = kernel.shape[1] // 2
    width = image.shape[0]
    height = image.shape[1]
    mask = (variance < threshold).astype(np.float)

    for iy in range(0, height):
        for ix in range(0, width):
            total = 0.
            conv_weight = 0.
            for cy in range(0, kernel.shape[1]):
                for cx in range(0, kernel.shape[0]):
                    x2 = ix + cx - hx
                    y2 = iy + cy - hy

                    if x2 >= 0 and x2 < width and y2 >= 0 and y2 < height:
                        total += image[x2, y2] * kernel[cx, cy] * mask[x2, y2]
                        conv_weight += kernel[cx, cy] * mask[x2, y2]

            if mask[ix, iy]:
                output[ix, iy] = total / conv_weight

    return output * mask

In [11]:
generateTileVersionB(kernel, img, var, threshold)

array([[ 0.        ,  0.        ,  0.        ,  3.33333333,  5.        ],
       [ 0.        ,  0.        ,  7.        ,  0.        ,  9.        ],
       [ 0.        , 11.        ,  0.        , 13.33333333, 13.8       ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 17.33333333],
       [20.33333333, 21.        , 22.        , 22.66666667,  0.        ]])

### Third rewrite
Line 18 is applying the kernel to the image and then mask. Or, equivalent, is applying the kernel to the masked image.
Line 19 is applying the kernel to the mask.

Lines 22 and 23 are dividing the convolved image by the convolved mask for non masked pixels.

So, equivalently:

In [14]:
def generateTileVersionC(kernel, image, variance, threshold=.5):
    output = np.zeros(image.shape)
    width = image.shape[0]
    height = image.shape[1]
    mask = (variance < threshold).astype(np.float)
    
    total = ndimage.convolve(image * mask, kernel, mode='constant')
    weight = ndimage.convolve(mask, kernel, mode='constant')

    for iy in range(0, height):
        for ix in range(0, width):
            if mask[ix, iy]:
                output[ix, iy] = total[ix, iy] / weight[ix, iy]

    return output

In [15]:
generateTileVersionC(kernel, img, var, threshold)

array([[ 0.        ,  0.        ,  0.        ,  3.33333333,  5.        ],
       [ 0.        ,  0.        ,  7.        ,  0.        ,  9.        ],
       [ 0.        , 11.        ,  0.        , 13.33333333, 13.8       ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 17.33333333],
       [20.33333333, 21.        , 22.        , 22.66666667,  0.        ]])